In [77]:
import gzip
import numpy as np
from collections import defaultdict
from sklearn.metrics import mean_squared_error

In [78]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [79]:
from random import shuffle
data = []
for l in readGz("train.json.gz"):
    data.append(l)

print(data[0])
shuffle(data)
print(data[0])

trainData, validData = np.array_split(data, 2)

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 5.0, 'businessID': 'B538084128', 'reviewText': 'Katrina- those five stars mean a lot to us:) Thank you for the high marks!', 'userID': 'U028657359', 'reviewTime': 'Apr 11, 2013', 'categories': ['Auto Repair Shop', 'Auto Parts Store', 'Brake Shop'], 'reviewHash': 'R436006326', 'unixReviewTime': 1365710096}


In [69]:
allRatings = []
userRatings = defaultdict(dict)
busRatings = defaultdict(dict)
globalAvg = 0.0

for d in trainData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    allRatings.append(rating)
    globalAvg += rating
    userRatings[user][bus] = rating
    busRatings[bus][user] = rating

numUsers = len(userRatings) #storing the number of unique users in training set
numBus = len(busRatings) #storing the number of unique users in training set

globalAvg = globalAvg/len(allRatings)

vallRatings = []
vuserRatings = defaultdict(dict)
vbusRatings = defaultdict(dict)

for d in validData:
    user, bus, rating = d['userID'], d['businessID'], d['rating']
    vallRatings.append(rating)
    vuserRatings[user][bus] = rating
    vbusRatings[bus][user] = rating

In [70]:
alpha = 0;
betaU = defaultdict(int)
betaI = defaultdict(int)
lowDimSize = 10
gammaU = defaultdict(lambda: [0 for x in range(lowDimSize)])
gammaI = defaultdict(lambda: [0 for x in range(lowDimSize)])
learningRate = 0.1
lamda = 10

def resetVars(lSize):
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize, globalAvg
    lowDimSize = lSize
    alpha = globalAvg;
    betaU = defaultdict(int)
    betaI = defaultdict(int)
    gammaU = defaultdict(lambda: [0 for x in range(lowDimSize)])
    gammaI = defaultdict(lambda: [0 for x in range(lowDimSize)])
    for user in userRatings:
        gammaU[user] = np.random.normal(0, size=lowDimSize)
    for bus in busRatings:
        gammaI[bus] = np.random.normal(0, size=lowDimSize)

resetVars(10)

In [75]:
predRatings = []

def predictValid():
    global predRatings
    global alpha, betaU, betaI, gammaU, gammaI, lowDimSize
    predRatings = []
    for d in validData:
        user, business = d['userID'], d['businessID']
        prediction = alpha + betaU[user] + betaI[business] + np.matmul(gammaU[user],np.transpose(gammaI[bus]))
        predRatings.append(prediction)
    MSE_alpha = mean_squared_error(vallRatings, predRatings)
    return MSE_alpha

def trainLatentFactor():
    global alpha, betaU, betaI, gammaU, gammaI, predRatings, learningRate, lamda
    for d in trainData:
        user, business, rating = d['userID'], d['businessID'], d['rating']
        prediction = alpha + betaU[user] + betaI[business] + np.matmul(gammaU[user], np.transpose(gammaI[business]))
        error = rating - prediction
        alpha += learningRate*(error - lamda*alpha)
        #print(alpha)
        betaU[user] += learningRate*(error - lamda*betaU[user])
        betaI[business] += learningRate*(error - lamda*betaI[business])
        #print("before")
        #print(betaU[user])
        gammaU[user][:] += learningRate*((error*gammaI[business][:]) - (lamda*gammaU[user][:]))
        #print(gammaU[user])
        gammaI[business][:] += learningRate*((error*gammaU[user][:]) - (lamda*gammaI[business][:]))
        #print(gammaI[business])
        #break
        
def train(i, lR, lamdat):
    global learningRate, lamda
    learningRate = lR
    lamda = lamdat
    for x in range(i):
        trainLatentFactor()
    print(predictValid())

In [76]:
resetVars(10)
train(10, 0.01, 10)
print(alpha)

11.4155364045
0.315307418791
